**Installation of all dependencies:**

In [ ]:
%pip install langchain_community langchain langchain_cohere langchain_groq langchain ollama transformers

  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached multiprocess-0.70.17-py311-none-any.whl (144 kB)
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.2 MB 817.9 kB/s eta 0:00:30
   - -------------------------------------- 1.0/25.2 MB 948.7 kB/s eta 0:00:26
   - -------------------------------------- 1.0/25.2 MB 948.7 kB/s eta 0:00:26
   -- ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 3.1.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.17 which is incompatible.
opentelemetry-proto 1.28.1 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.


In [1]:
import os
import langchain
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_groq import ChatGroq  
from langchain.llms import Ollama


**LLM (Llama3-8b-8192 / Llama3.2-3b)**

In [2]:
import os 

GROQ_API_KEY='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'
os.environ["GROQ_API_KEY"]='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'

In [3]:
llm= ChatGroq(
            model='llama3-8b-8192',
            temperature=0.6,
            api_key=os.getenv("GROQ_API_KEY")
        )


In [ ]:
llm =Ollama(model='llama3.2:3b') 

C:\Users\user\AppData\Local\Temp\ipykernel_2952\4284971636.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm =Ollama(model='llama3.2:3b')


**Testing LLMs**

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="Tell me about programming")
]
response = llm.invoke(messages)
print(response)

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])
chain = template | llm|StrOutputParser()
response = chain.invoke({"topic": "programming"})
print(response)


content="Programming! It's like trying to teach a computer to do a magic trick, but instead of making a rabbit appear, it's making your program appear to work correctly.\n\nWhy did the programmer quit his job? Because he didn't get arrays! (get a raise)\n\nBut seriously, programming is all about writing instructions that a computer can understand, using languages like Python, Java, or C++. It's like writing a recipe for your computer, telling it what to do step by step.\n\nHere's a joke to help you understand variables:\n\nWhy did the variable go to therapy? Because it was feeling undefined!\n\nAnd here's one about loops:\n\nWhy did the programmer break up with his girlfriend? Because he wanted to loop through all the other girls!\n\nProgramming can be challenging, but it's also super rewarding when you finally get something to work. And who knows, maybe one day you'll create the next big app or game!\n\nWant to hear another joke?" additional_kwargs={} response_metadata={'token_usage':

***Doccument Processing For RAG***

File upload 

In [4]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = r"C:\Users\user\Desktop\Alchemist AI Assignment\data"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 1 documents from the folder.


**Chunking**

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 115 chunks.


In [ ]:
print(splits[1])


page_content='dynasty, ruled over the Indian subcontinent from' metadata={'source': 'C:\\Users\\user\\Desktop\\Alchemist AI Assignment\\data\\The Mughal Empire.pdf', 'page': 0}


**Creating Embeddings**

In [7]:
#Cohere Embeddings

embeddings_model_name='embed-english-v2.0'
COHERE_API_KEY='30lAzH7VO1y9M00rSD5ns4gIIxicqxiMnCn1IycB'
import os
os.environ['COHERE_API_KEY']=COHERE_API_KEY 
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
        model=embeddings_model_name,
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\user\AppData\Local\sagemaker\sagemaker\config.yaml


In [ ]:
#Sentence Transformer Based Embeddings

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])  


C:\Users\user\AppData\Local\Temp\ipykernel_2464\154022173.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


KeyboardInterrupt: 

In [ ]:
embeddings

CohereEmbeddings(client=<cohere.client.Client object at 0x0000020A40F4D950>, async_client=<cohere.client.AsyncClient object at 0x0000020A42BC4A90>, model='embed-english-v2.0', truncate=None, cohere_api_key=SecretStr('**********'), embedding_types=['float'], max_retries=3, request_timeout=None, user_agent='langchain:partner', base_url=None)

**Vector Database Creation,Ingestion:ChromaDB**

In [ ]:
import chromadb

In [9]:
import pypika.dialects


In [ ]:
from chromadb.config import Settings
from chromadb import Client

# Initialize ChromaDB with persistence
settings = Settings(persist_directory="./chroma_db")
client = Client(settings=settings)

# List available collections
try:
    collections = client.list_collections()
    print(f"Collections: {collections}")
except Exception as e:
    print(f"Error listing collections: {e}")


Collections: []


In [ ]:
# Create or get a collection
try:
    collection = client.get_or_create_collection(name="my_collection")
    print("Collection created:", collection)
except Exception as e:
    print(f"Error creating collection: {e}")


Collection created: Collection(name=my_collection)


In [10]:
from langchain.vectorstores import Chroma
collection_name="my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [ ]:
query = "What was babar?"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'What was babar?'

Result 1:
Source: C:\Users\user\Desktop\Alchemist AI Assignment\data\The Mughal Empire.pdf
Content: Babur's grandson, Akbar the Great, is often

Result 2:
Source: C:\Users\user\Desktop\Alchemist AI Assignment\data\The Mughal Empire.pdf
Content: Babur's grandson, Akbar the Great, is often



In [11]:
#Normal Retriever

retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
# retriever_results = retriever.invoke("Who was Akbar?")
# print(retriever_results)


In [12]:
#Contextual Compression Retriever
from langchain.retrievers import ContextualCompressionRetriever
#from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import CohereRerank

#Compression
#Compressior=LLMChainExtractor.from_llm(llm)
compressor = CohereRerank(cohere_api_key=os.getenv('COHERE_API_KEY'))
#Contextual Compression
retriever=ContextualCompressionRetriever(base_compressor=compressor,
                                         base_retriever=retriever)



C:\Users\user\AppData\Local\Temp\ipykernel_10756\1361929948.py:8: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(cohere_api_key=os.getenv('COHERE_API_KEY'))


**Testing Simple RAG**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
question = "Who was Akbar?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: Who was Akbar?
Answer: Akbar the Great


In [ ]:
question = "Who was his son?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: Who was his son?
Answer: Aurangzeb


In [ ]:
question = "What was his religious policy?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: What was his religious policy?
Answer: The answer is: not mentioned. The context only mentions that Aurangzeb expanded the empire, but does not mention the religious policy of the father.


**History Aware Retriver based RAG**

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()


In [ ]:
print(contextualize_chain.invoke({"input": "Who was Akbar?", "chat_history": []}))


Who was Akbar, the Mughal Emperor?


In [ ]:
from langchain.chains import create_retrieval_chain

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question.Answer to the point and precise manner."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


**History Aware RAG chain**

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Who was Akbar?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Who was his Son?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Who was Akbar?
AI: Akbar was the grandson of Babur and the third Mughal Emperor of India. He is often credited with consolidating and expanding the Mughal Empire.

Human: Who was his Son?
AI: Akbar's son was Jahangir, who succeeded him as the fourth Mughal Emperor of India.


**Combining Every Block of Code**

In [ ]:

from langchain_core.documents import Document
import os
from dotenv import load_dotenv
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from typing import List

def create_chains(llm, retriever):
    """Create necessary chains for the RAG pipeline."""
    contextualize_ques_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        Given a chat history and the latest user question
        which might reference context in the chat history,
        formulate a standalone question which can be understood
        without the chat history. Do NOT answer the question,
        just reformulate it if needed and otherwise return it as is.
        """),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    contextualize_chain = contextualize_ques_prompt | llm | StrOutputParser()
    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_ques_prompt)

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a helpful RAG AI assistant. 
        Use the following context to answer the user's question. 
        Answer to the point,precise and in a very specific manner.
        If it is an out-of-context question, do not answer and say it is out of context.
        """),
        ("system", "Context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ])
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain


In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

def create_generic_prompt_chain(llm):
    """Create a chain to handle vague/generic inputs using FewShotPromptTemplate."""
    
    # Define a set of examples for FewShotPromptTemplate
    examples = [
    {
        "input": "Who was Akbar?",
        "output": "Akbar was the grandson of Babur and a great Mughal emperor."
    },
    {
        "input": "Tell me about Mughal empire.",
        "output": "Would you like to know more about Akbar or explore other aspects of the Mughal empire's history and achievements?"
    },
    {
        "input": "Explain photosynthesis.",
        "output": "Photosynthesis is the process by which plants convert sunlight into energy. Would you like details on the chemical process, its stages, or its significance in ecosystems?"
    },
    {
        "input": "What are planets?",
        "output": "Planets are celestial bodies orbiting stars. Are you asking about their formation, characteristics, or specific ones like Earth or Mars?"
    },
    {
        "input": "Tell me about Mars.",
        "output": "Mars is known as the Red Planet due to its iron oxide surface. Would you like to learn about its atmosphere, exploration missions, or potential for life?"
    }
]
    
    # Example template for FewShotPromptTemplate
    example_template = """
    User: {input}
    AI: {output}
    """
    example_prompt = PromptTemplate.from_template(example_template)
    
    # FewShotPromptTemplate
    generic_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="The following is a conversation between the AI and the user. The user's inputs are vague or generic. Use the chat history to understand the context and guide the user to provide more specific input.",
        suffix="Chat History:\n{chat_history}\nUser: {input}\nAI:",
        input_variables=["chat_history", "input"]
    )
    
    # Create the chain
    generic_chain =  generic_prompt | llm
    return generic_chain
 # Replace with your LLM instance
# generic_chain = create_generic_prompt_chain(llm)

# # Test the chain with a query
# query = "Tell me about Mughal empire."
# response = generic_chain.invoke(input=query)
# print(response)


In [ ]:
def create_generic_prompt_chain(llm):
    """Create a chain to handle vague/generic inputs using ChatPromptTemplate."""
    
    # Define a system message for context
    generic_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        The user's inputs are vague or generic. 
        Use the chat history to understand the context and guide the user 
        to provide more specific input.Don't answer the query but ask follow up question mainting the context .
        examples:
        "input": "Tell me about Mars.",
        "output": "Mars is known as the Red Planet due to its iron oxide surface. Would you like to learn about its atmosphere, exploration missions, or potential for life?"
    
        "input": "What are planets?",
        "output": "Planets are celestial bodies orbiting stars. Are you asking about their formation, characteristics, or specific ones like Earth or Mars?"
     
        """),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    
    # Combine the prompt and LLM
    generic_chain = generic_prompt | llm
    return generic_chain


In [ ]:
def get_last_k_messages(chat_history, k=1):
    """Get the last k messages from the chat history."""
    return chat_history[-k:] if len(chat_history) > k else chat_history


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate

def router_logic(llm, user_input):
    """Decides if the input is 'specific' or 'generic'."""
    router_prompt = PromptTemplate(
        input_variables=["input"],
        template="""
        Based on the input provided, classify it as either 'specific' or 'generic'.
        Input: {input}
        Respond with either 'specific' or 'generic' only.
        """
    )
    router_chain = router_prompt | llm
    classification = router_chain.invoke({"input": user_input})
    return classification.content.strip().lower()

def create_router_chain(llm, retriever,last_k_chat_history, user_input):
    """Combines RAG and generic chains with routing logic and returns the final answer."""
    # Create the history-aware RAG chain
    rag_chain = create_chains(llm, retriever)

    # Create the generic chain
    generic_chain = create_generic_prompt_chain(llm)
    
    # Determine whether input is 'specific' or 'generic'
    classification = router_logic(llm, user_input)
    print('classification: ',classification)
    if classification == "specific":
        # History-aware invocation for the RAG chain
        print('rag chain invoked')
        response = rag_chain.invoke({
            "input": user_input,
            "chat_history": last_k_chat_history
        })['answer']
    elif classification == "generic":
        # Direct invocation of the generic chain
        print('generic chain invoked')
        response = generic_chain.invoke({
            "input": user_input,
            "chat_history": last_k_chat_history
        })
    else:
        # Fallback for unclassified cases
        response = "I'm sorry, I couldn't classify your input. Please try again."

    return response




In [ ]:

user_input = "who was babur?."
response = create_router_chain(llm, retriever,chat_history, user_input)
print(response)

classification:  specific
rag chain invoked
Babur was a descendant of Timur and Genghis Khan, and the founder of the Mughal Empire in India. He was a Central Asian ruler who conquered India in the early 16th century and established the Mughal Empire.


In [ ]:

user_input = "Tell me anything interesting."
response = create_router_chain(llm, retriever,chat_history,user_input)
print(response)

classification:  generic
generic chain invoked
content='Jahangir! He was quite the interesting character. As the son of Akbar, he inherited the throne of the Mughal Empire, but his reign was marked by turmoil and power struggles. Did you know that Jahangir was known for his love of art, architecture, and literature? He even wrote his own poetry and was a patron of the arts.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 254, 'total_tokens': 330, 'completion_time': 0.063333333, 'prompt_time': 0.030050287, 'queue_time': 0.002686758000000001, 'total_time': 0.09338362}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None} id='run-7baf71b0-8da7-4efb-aef7-63dee8c60a68-0' usage_metadata={'input_tokens': 254, 'output_tokens': 76, 'total_tokens': 330}


**Setting Up SQLite**

In [ ]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [ ]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "Who made Taj Mahal?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "llama3-8b")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "Who was Babur?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "llama3-8b")
print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Who made Taj Mahal?
AI: The Taj Mahal was commissioned by Shah Jahan, the grandson of Akbar the Great, in memory of his beloved wife, Mumtaz Mahal.

Human: Who was Babur?
AI: Babur was the founder of the Mughal Empire in India. He was a descendant of Timur (also known as Tamerlane) and Genghis Khan, and was a Central Asian ruler who conquered India and established the Mughal Empire in 1526 after his victory at the Battle of Panipat.


RAGAS setup

In [13]:
!pip install -U -q langchain  ragas 

In [40]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [41]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

primary_qa_llm = llm

rag_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [42]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [43]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [44]:
question_generation_llm = llm

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [45]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. 
For each context, create a question that is specific to the context.
Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=splits[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [46]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
What was the primary source of revenue for the Mughal Empire during its peak in the 17th century?
context
{'page_content': 'The Mughal Empire, a prominent and influential', 'metadata': {'source': 'C:\\\\Users\\\\user\\\\Desktop\\\\Alchemist AI Assignment\\\\data\\\\The Mughal Empire.pdf', 'page': 0}}


In [47]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(splits[:7]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 7/7 [00:07<00:00,  1.08s/it]


In [48]:
qac_triples

[{'question': 'What was the name of the dynasty that ruled over the Indian subcontinent from 1526 to 1756, and who was its founder?',
  'context': Document(metadata={'source': 'C:\\Users\\user\\Desktop\\Alchemist AI Assignment\\data\\The Mughal Empire.pdf', 'page': 0}, page_content='dynasty, ruled over the Indian subcontinent from')},
 {'question': "What is the significance of the Mughal Empire's administrative capital, Fatehpur Sikri, in the context of Akbar's reign and the empire's expansion?",
  'context': Document(metadata={'source': 'C:\\Users\\user\\Desktop\\Alchemist AI Assignment\\data\\The Mughal Empire.pdf', 'page': 0}, page_content='from the')},
 {'question': 'What was the primary source of revenue for the Mughal Empire during the early 16th century, as mentioned in the text?',
  'context': Document(metadata={'source': 'C:\\Users\\user\\Desktop\\Alchemist AI Assignment\\data\\The Mughal Empire.pdf', 'page': 0}, page_content='early 16th century until the mid-18th century,')},

In [49]:
answer_generation_llm = llm

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [50]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
The Mughal Empire, founded by Babur


In [51]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 6/6 [00:05<00:00,  1.05it/s]


In [52]:
!pip install -q -U datasets

In [53]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [54]:
eval_dataset.to_csv("groundtruth_eval_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1949

In [55]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 6
})

In [56]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    # context_recall,
    # answer_correctness,
    # answer_similarity
)


#from ragas.metrics.critique import harmfulness
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
          "answer" : answer["response"].content,
          "contexts" : [context.page_content for context in answer["context"]],
          "ground_truths" : [row["ground_truth"]]
          }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset



In [57]:
for row in tqdm(eval_dataset):
        # Ensure the question is a string
        question = str(row["question"])
        print(question)

100%|██████████| 6/6 [00:00<?, ?it/s]

What was the name of the dynasty that ruled over the Indian subcontinent from 1526 to 1756, and who was its founder?
What is the significance of the Mughal Empire's administrative capital, Fatehpur Sikri, in the context of Akbar's reign and the empire's expansion?
What was the primary source of revenue for the Mughal Empire during the early 16th century, as mentioned in the text?
What was the primary reason for the decline of the Mughal Empire in the 18th century, despite its remnants lasting until the mid-19th century?
What was the primary source of revenue for the Mughal Empire during the mid-19th century, according to the text on page 0 of 'The Mughal Empire'?
What was the name of the founder who established the Mughal Empire in 1526?


In [58]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(rag_chain, eval_dataset)

100%|██████████| 6/6 [00:14<00:00,  2.34s/it]


In [59]:
basic_qa_ragas_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 6
})

In [60]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2858

In [61]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        # context_recall,
        # answer_correctness,
        # answer_similarity
    ]
  )
  return result

In [63]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

Exception raised in Job[9]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
Exception raised in Job[3]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
Exception raised in Job[5]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'ins

In [64]:
basic_qa_result

{'faithfulness': nan, 'answer_relevancy': nan}